In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df1 = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.info()

In [ ]:
df1.isnull().sum()

In [ ]:
df1.drop(columns=['keyword', 'location','id'], inplace = True)

In [ ]:
df1.head()

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

In [ ]:
from nltk.corpus import stopwords

In [ ]:
import string

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def transform(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
    text = y[:]#copying list
    y.clear()
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
    text = y[:]
    y.clear()
    for i in text:
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
df1['transform_text'] = df1['text'].apply(transform)

In [ ]:
df2 = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
df2.drop(columns=['keyword', 'id', 'location'], inplace = True)

In [ ]:
df2.isnull().sum()

In [ ]:
df2.shape

In [ ]:
df2['transform_text'] = df2['text'].apply(transform)

In [ ]:
df2.drop(columns=['text'], inplace = True)

In [ ]:
df2.head()

In [ ]:
df2.shape

#

In [ ]:
df1.head()

In [ ]:
df1.drop(columns=['text'], inplace = True)

In [ ]:
df1.shape

In [ ]:
df1 = pd.concat([df1, df2], axis = 0)

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.isnull().sum()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = CountVectorizer()
tf = TfidfVectorizer()

In [ ]:
X = tf.fit_transform(df1['transform_text'])

In [ ]:
X.shape

In [ ]:
X_tests = X[7613:10876].toarray()

In [ ]:
X_tests.shape

In [ ]:
XX = X[0:7613].toarray()

In [ ]:
XX.shape

In [ ]:
y = df1['target'].iloc[0:7613]

In [ ]:
y[7612]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(XX, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
random = RandomForestClassifier(n_estimators=110)

In [ ]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

In [ ]:
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
gnb.fit(X_train, y_train)
pred1 = gnb.predict(X_test)
print(classification_report(y_test, pred1))

In [ ]:
mnb.fit(X_train, y_train)
pred2 = mnb.predict(X_test)
print(classification_report(y_test, pred2))

In [ ]:
bnb.fit(X_train, y_train)
pred3 = bnb.predict(X_test)
print(classification_report(y_test, pred3))

In [ ]:
#random.fit(X_train, y_train)
#pred4 = random.predict(X_test)
#print(classification_report(y_test, pred4))

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
extra = ExtraTreesClassifier(n_estimators=100)

In [ ]:
#extra.fit(X_train, y_train)
#pred5 = extra.predict(X_test)
#print(classification_report(y_test, pred5))

In [ ]:
prediction = bnb.predict(X_tests)

In [ ]:
prediction

In [ ]:
final = pd.DataFrame(prediction, columns = ['target'])

In [ ]:
final['target'] = final['target'].apply(lambda x: int(x))

In [ ]:
submit = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [ ]:
submit.drop('target', axis = 1, inplace = True)

In [ ]:
submit = pd.concat([submit, final], axis = 1)

In [ ]:
submit.head()

In [ ]:
submit.to_csv('final1.csv', index = False)